Try TFIDF, Cosine Similarity, Text Embeddings, 


Graph Based
Text Rank

In [6]:
import os 
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
import heapq

import re
import nltk

In [18]:
stopwords = nltk.corpus.stopwords.words("english")

dirs = ['biorxiV_medrxiv', 'comm_use_subset', 'custom_license', 'noncomm_use_subset']

docs = []
for d in dirs:
    print(d)
    for file in tqdm(os.listdir(f"{d}/{d}")):
        filepath = f"{d}/{d}/{file}"
        j = json.load(open(filepath,'rb'))
        title = j['metadata']['title']
        try: 
            abstract = j['abstract'][0]['text']
        except:
            abstract = ''
            
        fulltext = ''
        for text in j['body_text']:
            fulltext += text['text'] + " "
        docs.append([title, abstract, fulltext])

  6%|▌         | 50/885 [00:00<00:01, 498.98it/s]

biorxiV_medrxiv


  1%|          | 50/9118 [00:00<00:18, 489.16it/s]

comm_use_subset


  0%|          | 47/16959 [00:00<00:36, 459.73it/s]

custom_license


  3%|▎         | 63/2353 [00:00<00:03, 622.86it/s]

noncomm_use_subset


100%|██████████| 2353/2353 [00:03<00:00, 595.86it/s]


In [19]:
df = pd.DataFrame(docs, columns = ['title', 'abstract', 'fulltext'])
fulltexts = df['fulltext'].values

# Text Embeddings

https://medium.com/jatana/unsupervised-text-summarization-using-sentence-embeddings-adb15ce83db1

In [ ]:
def clean_text(text):
    # Removing Square Brackets and Extra Spaces
    text = re.sub(r'\[[0-9]*\]',' ', text)
    text = re.sub(r'\s+',' ', text)
 
    text = re.sub(r'\{\{[\s\S]*?\}\}', '', text)

    # Remove doi links
    #text = re.sub(r'^https://$', '',text)
    return text

def clean_spchar_digs(text):
    # Removing special characters and digits
    text = re.sub('[^a-zA-Z]', ' ', text )
    text = re.sub(r'\s+', ' ', text)
    
    return text

def word_freq(formatted_text):
    #creates a dictionary of words as keys and frequency as values
    word_frequencies = {}
    for word in nltk.word_tokenize(formatted_text):
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1

    maximum_frequency = max(word_frequencies.values())
    #divides the values by the maximum frequency
    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word]/maximum_frequency)
    
    return word_frequencies

def sent_scores(sentence_list, word_frequencies):
    #uses the word frequencies to score the sentences by adding up the scores
    #of the words that make up the sentence
    sentence_scores = {}
    for sent in sentence_list:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) <60: #limits sentence to less than 60 words
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]
                    
    return sentence_scores

def get_summary(dirty_text):
    text = clean_text(dirty_text)
    formatted_text = clean_spchar_digs(text)

    sentence_list = nltk.sent_tokenize(text)

    word_frequencies = word_freq(formatted_text) 
    sentence_scores = sent_scores(sentence_list,word_frequencies)
    
    
    summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get) #first value is number highest scoring sentences to print
    summary = '\n\n '.join(summary_sentences)
    return summary

def get_improved_summary(searchlist):
    #get summary where all you have to do is provide the words you are searching for in a list
    covid_alias = ['CoV', 'COVID', 'Covid', 'corona virus', 'coronavirus', 'Coronavirus', 'Corona virus'] #depending on here the results w
    desired_sents = {}
    covid_sents = {}
    for text in full_text:
        for sentence in text.split('. '):
            for i in searchlist:
                if i.lower() in sentence.lower(): 
                    if sentence not in desired_sents.keys():
                        desired_sents[sentence] = sentence 
            for j in covid_alias:
                if j in sentence:
                    if sentence not in covid_sents.keys():
                        covid_sents[sentence] = sentence
                        
    desired_sents = set(desired_sents.keys())
    covid_sents = set(covid_sents.keys())
    desired_sents = list(desired_sents.intersection(covid_sents))
    desired_text = ''
    for x in desired_sents:
        desired_text += ' ' + x
    text = clean_text(desired_text)
    formatted_text = clean_spchar_digs(text)

    sentence_list = nltk.sent_tokenize(text)

    word_frequencies = word_freq(formatted_text) 
    sentence_scores = sent_scores(sentence_list,word_frequencies)
    
    
    summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get) #first value is number highest scoring sentences to print
    summary = '\n\n '.join(summary_sentences)
    return summary

# Cosine Similarity & Text Rank
https://towardsdatascience.com/understand-text-summarization-and-create-your-own-summarizer-in-python-b26a9f09fc70

In [20]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import networkx as nx
 
def return_clean_sentences(d_sents):
    sentences = []

    for i in d_sents:
            sentences.append(i.replace("[^a-zA-Z]", " "))
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


def generate_summary(fulltext,searchlist, top_n):
    stop_words = stopwords.words('english')
    summarize_text = []
    covid_alias = ['CoV', 'COVID', 'Covid', 'corona virus', 'coronavirus', 'Coronavirus', 'Corona virus'] #depending on here the results w
    desired_sents = {}
    covid_sents = {}
    for text in fulltexts:
        for sentence in text.split('. '):
            for i in searchlist:
                if i.lower() in sentence.lower(): #using .lower changes the results dramatically
                    if sentence not in desired_sents.keys():
                        desired_sents[sentence] = ''
            for j in covid_alias:
                if j in sentence:
                    if sentence not in covid_sents.keys():
                        covid_sents[sentence] = ''
    desired_sents = set(desired_sents.keys())
    covid_sents = set(covid_sents.keys())
    desired_sents = list(desired_sents.intersection(covid_sents))
                        
    # Step 1 - Read text anc split it
    sentences =  return_sentences(desired_sents)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize texr
    print("Summarize Text: \n", ". ".join(summarize_text))



In [22]:
# let's begin
slist = ['smoking', 'pulmonary']
generate_summary(fulltexts, slist,5)

Indexes of top ranked_sentence order are  [(0.0016236387292547806, 'Prospective longitudinal studies measuring objective parameters such as; pulmonary function testing, 6-min walk test and detection of depression, anxiety and post-traumatic stress disorder with correlation with health status, will provide more informative data to understanding of the overall long term outcomes of MERS-CoV infection'), (0.0016225206952418325, 'These include: (1) available animal models (eg, transduced mice, and transgenic mice, rabbits, rhesus macaques, marmosets, alpacas, and camels) might not mimic human disease; 7 (2) an immune correlate of protection has not been defined, and the protective immune response in natural infection is poorly understood, although both humoral and cellular responses are probably necessary for viral clearance; 8 (3) there is a theoretical risk of immune enhancement during MERS coronavirus infection after vaccination, possibly leading to immunopathological pulmonary eosinoph